In [1]:
import numpy
from numpy.linalg import inv
import csv
covariant_num = 1
XTX_files = ["XTX_withoutcovariant_1.tsv", "XTX_withoutcovariant_2.tsv"]
XTY_files = ["XTY_withoutcovariant_1.tsv", "XTY_withoutcovariant_2.tsv"]


In [2]:

class Linear_regression:
    XTX = {}
    XTY = {}
    beta = {}
    
    def __init__(self, XTX_files, XTY_files):
        for f in XTX_files:
            self.read_XTX(f)
        for f in XTY_files:
            self.read_XTY(f)

    def read_XTX(self, filename):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        for row in read_tsv:
            if row[0] == "locus":
                continue
            for i in range(2, len(row)-1):
                row[i] = int(row[i])
            XTX_entry = [[ 0 for i in range(covariant_num+1)] for j in range(covariant_num+1)]
            ii = 0
            for i in range(covariant_num+1):
                for j in range(i, covariant_num+1):
                    XTX_entry[i][j] = row[ii+2]
                    ii += 1
            for i in range(covariant_num+1):
                for j in range(i):
                    XTX_entry[i][j] = XTX_entry[j][i]
            npmatrix = numpy.array(XTX_entry)
            if row[0] in self.XTX:
                self.XTX[row[0]] = self.XTX[row[0]] + npmatrix
            else:
                self.XTX[row[0]] = npmatrix

    def read_XTY(self, filename):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        for row in read_tsv:
            if row[0] == "locus":
                continue
            for i in range(2, len(row)-1):
                row[i] = int(row[i])
            if row[0] in self.XTY:
                self.XTY[row[0]] = self.XTY[row[0]] + numpy.array(row[2:len(row)-1])
            else:
                self.XTY[row[0]] = numpy.array(row[2:len(row)-1])

    def export_beta(self):
        with open('beta.out', 'w') as f:
            sys.stdout = f
            for locu in self.XTX:
                try:
                    self.beta[locu] = inv(self.XTX[locu]).dot(self.XTY[locu])
                except numpy.linalg.LinAlgError as err:
                    if ('Singular matrix' in str(err)):
                        self.beta[locu] = 'NaN'
                    else:
                        raise
                print(locu, self.beta[locu])
        


In [3]:
gwas = Linear_regression(XTX_files, XTY_files)
gwas.export_beta()
